In [1]:
import cv2
import os, sys
import mediapipe as mp
from math import cos, sin
# from google.colab.patches import cv2_imshow
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

## A function to compute axes from angles

In [3]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [4]:
# choosing random image
random_file = 'image00053'
image = cv2.imread("../AFLW2000/image00070.jpg")
# loading the mat file to extract the 3 angels of the face (pitch,yaw,roll)
mat_file = sio.loadmat("../AFLW2000/image00070.mat")
# extracting the labels 3 angels
pose_para = mat_file["Pose_Para"][0][:3]
pitch = pose_para[0]
yaw = pose_para[1]
roll = pose_para[2]

cv2.imshow('HelloWorld', draw_axis(image,pitch,yaw,roll))
cv2.waitKey(2000) 
cv2.destroyWindow('HelloWorld')
# cv2_imshow()

## Get Features and Labels

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# For static images:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    min_detection_confidence=0.5) as face_mesh:
    
    All_Images = []
    # names = []
    angle1, angle2, angle3 = [], [], []
    for file_name in os.listdir("../AFLW2000"):
        if file_name.split(".")[-1].lower() in {"jpeg", "jpg", "png"}:
            image = cv2.imread("../AFLW2000/" + file_name)
            # Convert the BGR image to RGB before processing.
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if results.multi_face_landmarks != None:
                OneImage = []
                annotated_image = image.copy()

                for face in results.multi_face_landmarks:
                    #nose axes
                    nose_x = int(face.landmark[1].x * image.shape[1])
                    nose_y = int(face.landmark[1].y * image.shape[0])
                    # chin axes
                    chin_x = int(face.landmark[152].x * image.shape[1])
                    chin_y = int(face.landmark[152].y * image.shape[0])
                    # Max Dist
                    dist_x = nose_x - chin_x
                    dist_y = nose_y - chin_y

                    dist = np.sqrt(dist_x**2 + dist_y**2)

                    for landmark in face.landmark:
                        x = landmark.x
                        y = landmark.y

                        shape = image.shape 
                        relative_x = int(x * shape[1])
                        relative_y = int(y * shape[0])

                        OneImage.append((relative_x- nose_x) / dist)
                        OneImage.append((relative_y- nose_y) / dist)
                        cv2.circle(image, (relative_x, relative_y), radius=1, color=(225, 0, 100), thickness=1)

                All_Images.append(OneImage)
            # names.append(file_name)

                mat_file = sio.loadmat("../AFLW2000/" + file_name.split(".")[0] + '.mat')
                # extracting the labels 3 angels
                pose_para = mat_file["Pose_Para"][0][:3]
                pitch = pose_para[0]
                yaw = pose_para[1]
                roll = pose_para[2]

                angle1.append(pitch)
                angle2.append(yaw)
                angle3.append(roll)
    
# cv2_imshow(image)
# cv2.imshow('HelloWorld', draw_axis(image,pitch,yaw,roll))
# cv2.waitKey(0) 
# cv2.destroyWindow('HelloWorld')

# convert data into dataframe

In [6]:
columns = []
for i in range((len(All_Images[0])//2)):
    columns.append(('x' + str(i)))
    columns.append(('y' + str(i)))

print(len(columns))

936


In [7]:
df = pd.DataFrame(All_Images, columns = columns)
df['pitch'] = angle1
df['yaw'] = angle2
df['roll'] = angle3
# df['names'] = names

In [8]:
df.head()

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,y464,x465,y465,x466,y466,x467,y467,pitch,yaw,roll
0,-0.023515,0.258662,0.0,0.0,0.000000,0.047030,-0.058787,-0.399751,0.000000,-0.094059,...,-0.775987,0.270420,-0.728958,0.928833,-0.881804,0.987620,-0.928833,-0.399231,0.018227,0.085676
1,0.138885,0.277770,0.0,0.0,0.176763,0.101007,0.113633,-0.265144,0.000000,-0.088381,...,-0.454532,0.479784,-0.441907,0.959568,-0.517662,1.010072,-0.555540,0.470065,1.189533,0.300959
2,-0.022704,0.227038,0.0,0.0,-0.022704,0.045408,-0.113519,-0.317854,0.000000,-0.090815,...,-0.567596,0.068111,-0.544892,0.544892,-0.658411,0.567596,-0.681115,-0.184650,0.881137,-0.236852
3,0.180710,0.216852,0.0,0.0,0.108426,0.024095,-0.144568,-0.349372,-0.048189,-0.096379,...,-0.783076,0.108426,-0.734886,0.686697,-1.072211,0.722839,-1.132448,-0.175379,0.299208,-0.373374
4,0.078158,0.283321,0.0,0.0,0.029309,0.087927,-0.107467,-0.273552,-0.019539,-0.087927,...,-0.517794,0.058618,-0.498255,0.488485,-0.674110,0.527564,-0.693649,-0.026812,0.011965,-0.220662


### split data

In [9]:
X = df.drop(columns = ['pitch', 'yaw', 'roll'])
y = df.iloc[:,-3:]
y.head()

,pitch,yaw,roll
0,-0.399231,0.018227,0.085676
1,0.470065,1.189533,0.300959
2,-0.184650,0.881137,-0.236852
3,-0.175379,0.299208,-0.373374
4,-0.026812,0.011965,-0.220662


## Split data to train, val, test

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42, shuffle = True)

In [11]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

# Models
- LinearRegression
- SVR
- XGBoost
- GradientBoostingRegressor
- Ada Boost
- SGD Regressor

## Models for pitch

In [12]:
y_val.iloc[:, 0].shape

(278,)

In [13]:
from sklearn.linear_model import LinearRegression
lr1 = LinearRegression().fit(X_train, y_train.iloc[:,0])
lr1_pred = lr1.predict(X_val)

In [14]:
from sklearn.svm import SVR
svr1 = SVR().fit(X_train, y_train.iloc[:,0])
svr1_pred = svr1.predict(X_val)

In [15]:
from xgboost import XGBRegressor
xgb_reg1 = XGBRegressor().fit(X_train, y_train.iloc[:,0])
xgb1_pred = xgb_reg1.predict(X_val)

In [16]:
from sklearn.ensemble import GradientBoostingRegressor
gbr1 = GradientBoostingRegressor().fit(X_train, y_train.iloc[:,0])
gbr1_pred = gbr1.predict(X_val)

In [17]:
from sklearn.ensemble import AdaBoostRegressor
ada1 = AdaBoostRegressor().fit(X_train, y_train.iloc[:,0])
ada1_pred = ada1.predict(X_val)

In [18]:
from sklearn.linear_model import SGDRegressor
sgd1 = SGDRegressor().fit(X_train, y_train.iloc[:,0])
sgd1_pred = sgd1.predict(X_val)

In [19]:
print("linear regression score: ", lr1.score(X_val, y_val.iloc[:,0]))
print("SVM score: ", svr1.score(X_val, y_val.iloc[:,0]))
print("XGBoost: ", xgb_reg1.score(X_val, y_val.iloc[:,0]))
print("GBR: ", gbr1.score(X_val, y_val.iloc[:,0]))
print("Ada Boost: ", ada1.score(X_val, y_val.iloc[:,0]))
print("SGD Regressor: ", sgd1.score(X_val, y_val.iloc[:,0]))

linear regression score:  -5.292982059478126
SVM score:  0.6581026137255971
XGBoost:  -7.709833716597348
GBR:  -3.86304891358178
Ada Boost:  -7.813464354923312
SGD Regressor:  0.36527981714205837


In [20]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
print("mean square error of pitch (linear regression): ",mean_squared_error(y_val.iloc[:,0], lr1_pred))
print("mean square error of pitch (SVR): ", mean_squared_error(y_val.iloc[:,0], svr1_pred))
print("mean square error of pitch (XGBoost): ", mean_squared_error(y_val.iloc[:,0], xgb1_pred))
print("mean square error of pitch (Gradient boosting): ", mean_squared_error(y_val.iloc[:,0], gbr1_pred))
print("mean square error of pitch (Ada boost): ", mean_squared_error(y_val.iloc[:,0], ada1_pred))
print("mean square error of pitch (SGD Regressor): ", mean_squared_error(y_val.iloc[:,0], sgd1_pred))

mean square error of pitch (linear regression):  0.7012003893956599
mean square error of pitch (SVR):  0.03809618049488783
mean square error of pitch (XGBoost):  0.9704999531106221
mean square error of pitch (Gradient boosting):  0.5418689835159759
mean square error of pitch (Ada boost):  0.9820470770751727
mean square error of pitch (SGD Regressor):  0.07072418690705472


## Models for yaw

In [21]:
lr2 = LinearRegression().fit(X_train, y_train.iloc[:,1])
lr2_pred = lr2.predict(X_val)

In [22]:
svr2 = SVR().fit(X_train, y_train.iloc[:,1])
svr2_pred = svr2.predict(X_val)

In [23]:
xgb_reg2 = XGBRegressor().fit(X_train, y_train.iloc[:,1])
xgb2_pred = xgb_reg2.predict(X_val)

In [24]:
gbr2 = GradientBoostingRegressor().fit(X_train, y_train.iloc[:,1])
gbr2_pred = gbr2.predict(X_val)

In [25]:
ada2 = AdaBoostRegressor().fit(X_train, y_train.iloc[:,1])
ada2_pred = ada2.predict(X_val)

In [26]:
sgd2 = SGDRegressor().fit(X_train, y_train.iloc[:,1])
sgd2_pred = sgd2.predict(X_val)

In [27]:
print("linear regression score: ", lr2.score(X_val, y_val.iloc[:,1]))
print("SVM score: ", svr2.score(X_val, y_val.iloc[:,1]))
print("XGBoost: ", xgb_reg2.score(X_val, y_val.iloc[:,1]))
print("GBR: ", gbr2.score(X_val, y_val.iloc[:,1]))
print("Ada Boost: ", ada2.score(X_val, y_val.iloc[:,1]))
print("SGD Regressor: ", sgd2.score(X_val, y_val.iloc[:,1]))

linear regression score:  0.5285424825008779
SVM score:  0.9136129877750628
XGBoost:  0.4647156899060283
GBR:  0.7588056709475324
Ada Boost:  0.8535445983861959
SGD Regressor:  0.8941290337047769


In [28]:
print("mean square error of yaw (linear regression): ",mean_squared_error(y_val.iloc[:,1], lr2_pred))
print("mean square error of yaw (SVR): ", mean_squared_error(y_val.iloc[:,1], svr2_pred))
print("mean square error of yaw (XGBoost): ", mean_squared_error(y_val.iloc[:,1], xgb2_pred))
print("mean square error of yaw (Gradient boosting): ", mean_squared_error(y_val.iloc[:,1], gbr2_pred))
print("mean square error of yaw (Ada boost): ", mean_squared_error(y_val.iloc[:,1], ada2_pred))
print("mean square error of yaw (SGD Regressor): ", mean_squared_error(y_val.iloc[:,1], sgd2_pred))

mean square error of yaw (linear regression):  0.13749639368039415
mean square error of yaw (SVR):  0.02519400412737102
mean square error of yaw (XGBoost):  0.15611091031495952
mean square error of yaw (Gradient boosting):  0.07034218182964569
mean square error of yaw (Ada boost):  0.042712415879442064
mean square error of yaw (SGD Regressor):  0.030876326117928208


## Models for roll

In [29]:
lr3 = LinearRegression().fit(X_train, y_train.iloc[:,2])
lr3_pred = lr3.predict(X_val)

In [30]:
svr3 = SVR().fit(X_train, y_train.iloc[:,2])
svr3_pred = svr3.predict(X_val)

In [31]:
xgb_reg3 = XGBRegressor().fit(X_train, y_train.iloc[:,2])
xgb3_pred = xgb_reg3.predict(X_val)

In [32]:
gbr3 = GradientBoostingRegressor().fit(X_train, y_train.iloc[:,2])
gbr3_pred = gbr3.predict(X_val)

In [33]:
ada3 = AdaBoostRegressor().fit(X_train, y_train.iloc[:,2])
ada3_pred = ada3.predict(X_val)

In [34]:
sgd3 = SGDRegressor().fit(X_train, y_train.iloc[:,2])
sgd3_pred = sgd3.predict(X_val)

In [35]:
print("linear regression score: ", lr3.score(X_val, y_val.iloc[:,2]))
print("SVM score: ", svr3.score(X_val, y_val.iloc[:,2]))
print("XGBoost: ", xgb_reg3.score(X_val, y_val.iloc[:,2]))
print("GBR: ", gbr3.score(X_val, y_val.iloc[:,2]))
print("Ada Boost: ", ada3.score(X_val, y_val.iloc[:,2]))
print("SGD Regression:", sgd3.score(X_val, y_val.iloc[:,2]))

linear regression score:  -2.094002735190261
SVM score:  0.7463368859565688
XGBoost:  -4.811827648705032
GBR:  -5.2325441782860755
Ada Boost:  -4.7576351180030985
SGD Regression: 0.6291665061362139


In [36]:
print("mean square error of roll (linear regression): ",mean_squared_error(y_val.iloc[:,2], lr3_pred))
print("mean square error of roll (SVR): ", mean_squared_error(y_val.iloc[:,2], svr3_pred))
print("mean square error of roll (XGBoost): ", mean_squared_error(y_val.iloc[:,2], xgb3_pred))
print("mean square error of roll (Gradient boosting): ", mean_squared_error(y_val.iloc[:,2], gbr3_pred))
print("mean square error of roll (Ada boost): ", mean_squared_error(y_val.iloc[:,2], ada3_pred))
print("mean square error of roll (SGD Regressor): ", mean_squared_error(y_val.iloc[:,2], sgd3_pred))

mean square error of roll (linear regression):  0.4988561217803599
mean square error of roll (SVR):  0.04089892871495919
mean square error of roll (XGBoost):  0.9370598701524981
mean square error of roll (Gradient boosting):  1.0048933642630926
mean square error of roll (Ada boost):  0.9283222322092766
mean square error of roll (SGD Regressor):  0.05979069005696059


# using Grid Search

### Grid Search for pitch

In [37]:
from sklearn.model_selection import GridSearchCV

svm_parameters = {'kernel': ['rbf'],
                  'C': [0.01, 0.1, 1 , 10, 100],
                  'gamma': ['scale', 0.01, 0.1, 1, 10, 100]
                }
# use gridsearch to find the best hyperparameters 
svm1 = SVR()
svm_gs1 = GridSearchCV(svm1, svm_parameters).fit(X_train, y_train.iloc[:,0])

# git the best model
svm_winner1 = svm_gs1.best_estimator_
svm_winner1
svm_winner1.score(X_val, y_val.iloc[:,0])

0.6514618454710264

In [38]:
svm_gs1.best_score_

0.3915604378038203

In [39]:
svm_winner1

SVR(C=10)

### Grid Search for yaw

In [40]:
svm_parameters = {'kernel': ['rbf'],
                  'C': [0.01, 0.1, 1 , 10, 100],
                  'gamma': ['scale', 0.01, 0.1, 1, 10, 100]
                }
# use gridsearch to find the best hyperparameters 
svm2 = SVR()
svm_gs2 = GridSearchCV(svm2, svm_parameters).fit(X_train, y_train.iloc[:,1])

# git the best model
svm_winner2 = svm_gs2.best_estimator_
svm_winner2

svm_winner2.score(X_val, y_val.iloc[:,1])

0.9136129877750628

In [41]:
svm_gs2.best_score_

0.8515986503524479

In [42]:
svm_winner2

SVR(C=1)

### Grid Search for roll

In [43]:
svm_parameters = {'kernel': ['rbf'],
                  'C': [0.01, 0.1, 1 , 10, 100],
                  'gamma': ['scale', 0.01, 0.1, 1, 10, 100]
                }
# use gridsearch to find the best hyperparameters 
svm3 = SVR()
svm_gs3 = GridSearchCV(svm3, svm_parameters).fit(X_train, y_train.iloc[:,2])

# git the best model
svm_winner3 = svm_gs3.best_estimator_
svm_winner3

svm_winner3.score(X_val, y_val.iloc[:,2])

0.7540155043956163

In [44]:
svm_gs3.best_score_

0.5073445060844237

In [45]:
svm_winner3

SVR(C=1, gamma=0.01)

In [46]:
print("best model score for pitch",svm_winner1.score(X_test, y_test.iloc[:,0]))
print("best model score for yaw",svm_winner2.score(X_test, y_test.iloc[:,1]))
print("best model score for roll",svm_winner3.score(X_test, y_test.iloc[:,2]))

best model score for pitch 0.6616503665381653
best model score for yaw 0.9231393279877682
best model score for roll 0.5719187975475697


In [47]:
svr_pred = svm_winner1.predict(X_test)
mean_squared_error(y_test.iloc[:,0], svr_pred)

0.02544750066978002

In [48]:
svr_pred = svm_winner2.predict(X_test)
mean_squared_error(y_test.iloc[:,1], svr_pred)

0.020466241525005393

In [49]:
svr_pred = svm_winner3.predict(X_test)
mean_squared_error(y_test.iloc[:,2], svr_pred)

0.05294577975714337

## Saving models

In [50]:
import joblib

my_model_pitch = "my_model_pitch.pkl"
joblib.dump(svm_winner1, my_model_pitch)

my_model_yaw = "my_model_yaw.pkl"
joblib.dump(svm_winner2, my_model_yaw)

my_model_roll = "my_model_roll.pkl"
joblib.dump(svm_winner3, my_model_roll)


['my_model_roll.pkl']

In [51]:
import joblib
svm_model1 = joblib.load("my_model_pitch.pkl")

svm_model2 = joblib.load("my_model_yaw.pkl")

svm_model3 = joblib.load("my_model_roll.pkl")

In [52]:
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# For static images:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    min_detection_confidence=0.5) as face_mesh:

    All_Images = []
    image = cv2.imread("../last version/asmaaa.jpg")
    # Convert the BGR image to RGB before processing.
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_face_landmarks != None:

        OneImage = []
        annotated_image = image.copy()

    for face in results.multi_face_landmarks:

        #nose axes
        nose_x = int(face.landmark[1].x * image.shape[1])
        nose_y = int(face.landmark[1].y * image.shape[0])

        # chin axes
        chin_x = int(face.landmark[152].x * image.shape[1])
        chin_y = int(face.landmark[152].y * image.shape[0])

        # Max Dist
        dist_x = nose_x - chin_x
        dist_y = nose_y - chin_y

        dist = np.sqrt(dist_x**2 + dist_y**2)

        for landmark in face.landmark:
            x = landmark.x
            y = landmark.y

            shape = image.shape 

            relative_x = int(x * shape[1])
            relative_y = int(y * shape[0])

            OneImage.append((relative_x- nose_x) / dist)
            OneImage.append((relative_y- nose_y) / dist)

            cv2.circle(image, (relative_x, relative_y), radius=1, color=(225, 0, 100), thickness=1)
    All_Images.append(OneImage)
      
#cv2_imshow(OneImage)

# predict axes on an image 

In [53]:
image = cv2.imread("../last version/asmaaa.jpg")

svr1_pred = svm_model1.predict(All_Images)
svr2_pred = svm_model2.predict(All_Images)
svr3_pred = svm_model3.predict(All_Images)

cv2.imshow('HelloWorld', draw_axis(image, svr1_pred, svr2_pred, svr3_pred, tdx=nose_x, tdy=nose_y))
cv2.waitKey(10000) 
cv2.destroyWindow('HelloWorld')
# cv2_imshow(draw_axis(image, svr1_pred, svr2_pred, svr3_pred, tdx=nose_x, tdy=nose_y))

C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


# predict axes on a video

### download video from local file system

In [58]:
cam = cv2.VideoCapture("../mr ben3.mp4")


width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

cam.set(cv2.CAP_PROP_FPS, 30)
cam.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

size = (width, height)
i=0 #frame counter
frameTime = 1 

# Below VideoWriter object will create
# a frame of above defined The output 
# is stored in 'file_name.avi' file.
result = cv2.VideoWriter('mr ben.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         15, size)

All_Images = []
with mp_face_mesh.FaceMesh(static_image_mode= False) as mesh_faces:
    while cam.isOpened():
        
        OneImage = []
        success = cam.grab() 
        i=i+1 #increment counter
        
        if i % 3 == 0: # display only one third of the frames, you can change this parameter according to your needs
            
            success, frame = cam.retrieve() #decode frame
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                break
                
            # convert BGR image to RGB  before processing
            results = mesh_faces.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))    
            if results.multi_face_landmarks != None:
                for face in results.multi_face_landmarks: 
                    
                    print(len(face.landmark))
                    
                    # Nose 
                    nose_x = int(face.landmark[1].x * frame.shape[1])
                    nose_y = int(face.landmark[1].y * frame.shape[0])
                    
                    # chin
                    chin_x = int(face.landmark[152].x * frame.shape[1])
                    chin_y = int(face.landmark[152].y * frame.shape[0])
                    
                    # Max Dist
                    dist_x = nose_x - chin_x
                    dist_y = nose_y - chin_y
                    
                    dist = np.sqrt(dist_x**2 + dist_y**2)
                    
                    for landmark in face.landmark:  
                        
                        x = landmark.x
                        y = landmark.y
                        
                        shape = frame.shape
                        
                        relative_x = int(x * shape[1])
                        relative_y = int(y * shape[0])
                        
                        OneImage.append((relative_x- nose_x) / dist)
                        OneImage.append((relative_y- nose_y) / dist)
                        
                All_Images.append(OneImage)                     
                if cv2.waitKey(frameTime) & 0xff == ord('q'):
                    break
                    
                svr1_pred = svm_model1.predict(np.array(OneImage).reshape(1,-1))
                svr2_pred = svm_model2.predict(np.array(OneImage).reshape(1,-1))
                svr3_pred = svm_model3.predict(np.array(OneImage).reshape(1,-1))
                
                cv2.imshow('HelloWorld', draw_axis(frame, svr1_pred, svr2_pred, svr3_pred, tdx=nose_x, tdy=nose_y, size = 100))
                
                if success == True: 
                    
                    result.write(frame)

                    
cam.release()
cv2.destroyAllWindows()
result.release()

468
468
468
468
468
468
468


C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa 

468
468
468
468
468
468
468
468


C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa 

468
468
468
468
468
468
468
468
468
468


...

C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa 

468
468
468
468
468
468
468
468
468


C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa 

468
468
468
468
468
468
468
468
468
468
468
468
468
Ignoring empty camera frame.


C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa mohammed\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\asmaa 

In [55]:
# from google.colab import files
# files.download('asmaa2.avi')